In [1]:
from reaktoro import *
from reaktplot import *
import pandas as pd
import numpy as np

In [2]:

def define_system(mineral): # Create the database
    db = SupcrtDatabase("supcrtbl")

    # Create an aqueous phase automatically selecting all species with provided elements
    aqueousphase = AqueousPhase(speciate("H O C Ca Mg K Cl Na S N"))
    aqueousphase.set(ActivityModelPitzer())

    # Create a gaseous phase
    gaseousphase = GaseousPhase("CO2(g)")
    gaseousphase.set(ActivityModelPengRobinson())

    # Create a mineral phase
    mineralphase = MineralPhase(mineral)

    # Create the chemical system
    system = ChemicalSystem(db, aqueousphase, gaseousphase, mineralphase)

        # Define equilibrium specs
    specs = EquilibriumSpecs (system)
    specs.temperature()
    specs.pressure()
    specs.pH()
    specs.charge()
    specs.openTo("Cl-")

    # Define conditions to be satisfied at the chemical equilibrium state
    conditions = EquilibriumConditions(specs)
    conditions.charge(0.0, "mol") # to make sure the mixture is charge neutral

    # Define the equilibrium solver
    solver = EquilibriumSolver(specs)

    # Define aqueous properties
    aprops = AqueousProps(system)

    return system, conditions, solver, aprops



In [3]:
def seawater(system,mult=1):

    state = ChemicalState(system)
    # Seawater composition
    state.setTemperature(25, "celsius")
    state.setPressure(1.0, "bar")

    state.add("H2O(aq)",     1.0*mult, "kg")
    state.add("Ca+2"   ,   400*mult, "mg")
    state.add("Mg+2"   ,  1262*mult, "mg")
    state.add("Na+"    , 10556*mult, "mg")
    state.add("K+"     ,   380*mult, "mg")
    state.add("Cl-"    , 18977.2*mult, "mg")
    state.add("HCO3-"  ,   140*mult, "mg")
    state.add("SO4-2"  ,  2646*mult, "mg")

    return state

In [4]:

def solubility_pH_change(df, Conc, conditions, solver, aprops, state, T, pH,mineral, n0mineral):

    conditions.temperature(T, "celsius")
    conditions.pH(pH)
    # conditions.charge(0.0)

    res = solver.solve(state, conditions)
    # assert res.succeeded()

    aprops.update(state)
    chem_add = float(state.equilibrium().titrantAmount("H+"))

    state.set(mineral, n0mineral, "mol")
    res = solver.solve(state, conditions)
    # assert res.succeeded()

    # Update aqueous properties
    aprops.update(state)

    nMieral = float(state.speciesAmount(mineral))

    df.loc[len(df)] = [Conc, T, float(aprops.pH()), chem_add, n0mineral - nMieral]

In [5]:
mineral = "Calcite"
n0mineral = 15

df_water_calcite = pd.DataFrame(columns=["Conc","T", "pH", "titrantAmount",  "deltaCalcite"])
temperatures = np.arange(20,110,10)
pHs = [6,7,8,9,10]
conc = np.arange(1,8,1)

system, conditions, solver, aprops = define_system(mineral)
for i in conc:
    state = seawater(system,mult=i)
    [solubility_pH_change(df_water_calcite, i, conditions, solver, aprops, state, T, p, mineral, n0mineral) for T in temperatures for p in pHs]

In [6]:
mineral = "Anhydrite"
n0mineral = 15

df_water_anhydrite = pd.DataFrame(columns=["Conc","T", "pH", "titrantAmount",  "deltaAnhydrite"])
temperatures = np.arange(20,110,10)
pHs = [6,7,8,9,10]
conc = np.arange(1,8,1)

system, conditions, solver, aprops = define_system(mineral)
for i in conc:
    state = seawater(system,mult=i)
    [solubility_pH_change(df_water_anhydrite, i, conditions, solver, aprops, state, T, p, mineral, n0mineral) for T in temperatures for p in pHs]

In [9]:
df_sol = pd.concat([df_water_calcite, df_water_anhydrite["deltaAnhydrite"]], axis=1)

print(df_sol)



     Conc      T    pH  titrantAmount  deltaCalcite  deltaAnhydrite
0     1.0   20.0   6.0       0.001306  3.485286e-02    6.267075e-02
1     1.0   20.0   7.0      -0.051426 -2.504663e-13   -7.460699e-14
2     1.0   20.0   8.0      -0.005571  1.776357e-15    1.682405e-10
3     1.0   20.0   9.0      -0.000741 -5.329071e-15   -1.389218e-10
4     1.0   20.0  10.0      -0.000315  0.000000e+00   -1.739053e-12
..    ...    ...   ...            ...           ...             ...
310   7.0  100.0   6.0       0.261312 -2.602363e-12    2.201155e-10
311   7.0  100.0   7.0      -0.104614  3.552714e-15   -8.881784e-15
312   7.0  100.0   8.0      -0.010729 -6.927792e-14    8.881784e-15
313   7.0  100.0   9.0      -0.026601 -1.776357e-15   -1.755396e-11
314   7.0  100.0  10.0      -0.188583 -1.776357e-15   -4.334311e-13

[315 rows x 6 columns]


In [11]:
df_sol.to_csv("solubility_data.csv")